In [1]:
import thermopyl as th
from thermopyl import thermoml_lib
import cirpy
import numpy as np
import pandas as pd

import functools
from functools import reduce
from os.path import expanduser
from joblib import Memory

"""
Получить мольные доли компонентов и давление. CO2 и n-Decan. CO2 обычно идет вторым.
Построить зависимость мольного соотношения от давления.
"""

'\nПолучить мольные доли компонентов и давление. CO2 и n-Decan. CO2 обычно идет вторым.\nПостроить зависимость мольного соотношения от давления.\n'

In [2]:
home = expanduser("~")

In [3]:
desired_components = ['carbon dioxide', 'n-dodecane']

In [4]:
df = th.pandas_dataframe()

In [5]:
mem = Memory(location=home+"/.thermoml/")

In [6]:
@mem.cache
def resolve_cached(x, rtype):
   return cirpy.resolve(x, rtype)

In [7]:
dt = list(df.columns)

In [8]:
experiments=['Mole fraction']
ind_list = [df[exp].dropna().index for exp in experiments]
ind_list

[Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
        ...
        91188, 91189, 91190, 91191, 91192, 91193, 91194, 91195, 91196, 91197],
       dtype='int64', length=86642)]

In [9]:
ind = reduce(lambda x, y: x.union(y), ind_list)
ind

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       91188, 91189, 91190, 91191, 91192, 91193, 91194, 91195, 91196, 91197],
      dtype='int64', length=86642)

In [10]:
df_molefrac = df.iloc[ind]

In [11]:
name_to_formula = pd.read_parquet("~/.thermoml/compound_name_to_formula.pq").dropna()
name_to_formula

,formula
carbon dioxide,CO2
2-hydroxyethylammonium acetate,C4H11NO3
"2,2'-azanediylbis(ethanol) acetate",C6H15NO4
N-methyldiethanolammonium acetate,C7H17NO4
2-hydroxyethylammonium lactate,C5H13NO4
...,...
4-pyridinecarboxamide,C6H6N2O
sodium bicarbonate,CHNaO3
bis(benzoxazolyl-2-methyl) sulfide,C16H12N2O2S
1-methyl-1-propylpyrrolidinium bis[(trifluoromethyl)sulfonyl]imide,C10H18F6N2O4S2


In [12]:
df_molefrac['n_components'] = df_molefrac.components.apply(
    lambda x: len(x.split('__'))
)
df_binary = df_molefrac[df_molefrac.n_components == 2]

/tmp/ipykernel_29847/876262358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_molefrac['n_components'] = df_molefrac.components.apply(


In [14]:
df_binary.dropna(axis=1, how='all', inplace=True)
df_binary

/tmp/ipykernel_29847/883037651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binary.dropna(axis=1, how='all', inplace=True)


,filename,components,component_1_inchi,component_2_inchi,"Pressure, kPa","Temperature, K",Mole fraction,phase,component_3_inchi,component_4_inchi,...,"Viscosity, Pa*s",Mole fraction metadata,"Thermal diffusivity, m2/s","Excess molar enthalpy (molar enthalpy of mixing), kJ/mol","Critical pressure, kPa","Binary diffusion coefficient, m2/s","Henry's Law constant (mole fraction scale), kPa","Isobaric coefficient of expansion, 1/K",Activity coefficient,n_components
0,/home/mikhail/.thermoml/j.jct.2009.04.003.xml,carbon dioxide__2-hydroxyethylammonium acetate,InChI=1S/CO2/c2-1-3,"InChI=1S/C2H7NO.C2H4O2/c3-1-2-4;1-2(3)4/h4H,1-...",116.0,298.15,0.012000,Liquid,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,/home/mikhail/.thermoml/j.jct.2009.04.003.xml,carbon dioxide__2-hydroxyethylammonium acetate,InChI=1S/CO2/c2-1-3,"InChI=1S/C2H7NO.C2H4O2/c3-1-2-4;1-2(3)4/h4H,1-...",352.0,298.15,0.035900,Liquid,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,/home/mikhail/.thermoml/j.jct.2009.04.003.xml,carbon dioxide__2-hydroxyethylammonium acetate,InChI=1S/CO2/c2-1-3,"InChI=1S/C2H7NO.C2H4O2/c3-1-2-4;1-2(3)4/h4H,1-...",546.0,298.15,0.055600,Liquid,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,/home/mikhail/.thermoml/j.jct.2009.04.003.xml,carbon dioxide__2-hydroxyethylammonium acetate,InChI=1S/CO2/c2-1-3,"InChI=1S/C2H7NO.C2H4O2/c3-1-2-4;1-2(3)4/h4H,1-...",894.0,298.15,0.091200,Liquid,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,/home/mikhail/.thermoml/j.jct.2009.04.003.xml,carbon dioxide__2-hydroxyethylammonium acetate,InChI=1S/CO2/c2-1-3,"InChI=1S/C2H7NO.C2H4O2/c3-1-2-4;1-2(3)4/h4H,1-...",1189.0,298.15,0.120200,Liquid,None,None,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91130,/home/mikhail/.thermoml/je0505265.xml,water__erythromycin A dihydrate,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",101.0,303.15,0.000042,Liquid,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C3H8O/c1-3(2)4/h3-4H,1-2H3",...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
91131,/home/mikhail/.thermoml/je0505265.xml,water__erythromycin A dihydrate,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",101.0,308.00,0.000040,Liquid,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C3H8O/c1-3(2)4/h3-4H,1-2H3",...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
91132,/home/mikhail/.thermoml/je0505265.xml,water__erythromycin A dihydrate,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",101.0,313.10,0.000038,Liquid,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C3H8O/c1-3(2)4/h3-4H,1-2H3",...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
91133,/home/mikhail/.thermoml/je0505265.xml,water__erythromycin A dihydrate,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",101.0,318.00,0.000035,Liquid,"InChI=1S/C37H67NO13.2H2O/c1-14-25-37(10,45)30(...","InChI=1S/C3H8O/c1-3(2)4/h3-4H,1-2H3",...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [19]:
'__'.join(desired_components)

'carbon dioxide__dodecane'

In [26]:
df_mycomponents = df_binary[df_binary.component_2_inchi == 'InChI=1S/CO2/c2-1-3']
df_mycomponents = df_mycomponents[df_mycomponents.component_1_inchi == 'InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,1-2H3']
df_mycomponents

,filename,components,component_1_inchi,component_2_inchi,"Pressure, kPa","Temperature, K",Mole fraction,phase,component_3_inchi,component_4_inchi,...,"Isobaric coefficient of expansion, 1/K","Heat capacity at constant pressure per volume, J/K/m3","Molar heat capacity at constant pressure, J/K/mol","Frequency, MHz","Kinematic viscosity, m2/s",(Relative) activity,"Amount density, mol/m3","Tracer diffusion coefficient, m2/s","Thermal diffusivity, m2/s",n_components
34167,/home/vvp/.thermoml/j.fluid.2012.09.037.xml,decane__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,1120.0,373.20,0.0750,Liquid,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
34168,/home/vvp/.thermoml/j.fluid.2012.09.037.xml,decane__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,1950.0,373.20,0.1240,Liquid,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
34169,/home/vvp/.thermoml/j.fluid.2012.09.037.xml,decane__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,3040.0,373.20,0.1920,Liquid,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
34170,/home/vvp/.thermoml/j.fluid.2012.09.037.xml,decane__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,4010.0,373.20,0.2460,Liquid,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
34171,/home/vvp/.thermoml/j.fluid.2012.09.037.xml,decane__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,5110.0,373.20,0.3130,Liquid,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250585,/home/vvp/.thermoml/j.fluid.2016.03.010.xml,toluene__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,NaN,343.10,0.8788,Liquid,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
250586,/home/vvp/.thermoml/j.fluid.2016.03.010.xml,toluene__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,NaN,343.10,0.8997,Liquid,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
250587,/home/vvp/.thermoml/j.fluid.2016.03.010.xml,toluene__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,NaN,343.00,0.9217,Liquid,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
250588,/home/vvp/.thermoml/j.fluid.2016.03.010.xml,toluene__carbon dioxide,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,NaN,303.56,0.9211,Liquid,"InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...",InChI=1S/CO2/c2-1-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [29]:
df_mycomponents.iloc[-1]

filename                                                          /home/vvp/.thermoml/j.fluid.2016.03.010.xml
components                                                                            toluene__carbon dioxide
component_1_inchi                                           InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...
component_2_inchi                                                                         InChI=1S/CO2/c2-1-3
Pressure, kPa                                                                                             NaN
Temperature, K                                                                                         312.39
Mole fraction                                                                                          0.9211
phase                                                                                                  Liquid
component_3_inchi                                           InChI=1S/C10H22/c1-3-5-7-9-10-8-6-4-2/h3-10H2,...
component_